In [1]:
from datetime import datetime
import os
import numpy as np
import pandas as pd
import tushare as ts
import time
from tqdm import tqdm

In [2]:
'''
RunParams类：
    start_date: datetime，运行开始日期
    end_date: datetime，运行结束日期
    type: str，运行类型（回测/模拟）
'''
class RunParams:
    def __init__(self) -> None:
        self.start_date = None
        self.end_date = None
        self.run_type = None
        self.commission_rate = None
        self.algo_type = None
        self.algo_params = None
        self.order_type = None
        

'''
Portfolio类：
    position: dict，记录当前持仓
    available_cash: float，记录当前可用资金
    total_value: float，记录当前总资产
    returns: float，记录当前收益
    starting_cash: float，记录初始资金
    position_value: float，记录当前持仓市值
'''
class Portfolio:
    def __init__(self, starting_cash=100000000):
        self.positions = {}
        self.available_cash = starting_cash
        self.total_value = starting_cash
        self.returns = 0
        self.starting_cash = starting_cash
        self.position_value = 0

'''
Context类：
    portfolio: Portfolio，记录当前持仓信息
    current_dt: datetime，记录当前时间
    previous_date: datetime，记录上一个交易日
    run_params: RunParams，运行参数
    universe: list，股票池
'''
class Context:
    def __init__(self, run_params, portfolio):
        self.run_params = run_params
        self.portfolio = portfolio
        self.current_dt = run_params.start_date
        self.previous_dt = run_params.start_date
        self.universe = None
        self.benchmark = None
        self.trade_book = {}
        self.target_book = pd.DataFrame(columns=self.universe)
        self.position_book = None
        self.total_value_book = pd.DataFrame(columns=['total_value'])
        self.tca_book = {}

'''
全局变量G类：
    用于存储全局变量
'''
class G:
    pass


In [3]:
portfolio = Portfolio(starting_cash=1000000)
run_params = RunParams()
context = Context(run_params, portfolio)
g = G()

In [4]:
'''
数据获取相关函数
'''
# 获取tick数据
def get_tick_data(stock_codes):
    ts.set_token('567cc6a3f980227a5844d977ee1e53b96555778bb3d038a0613f7699')
    df = ts.realtime_quote(ts_code=stock_codes)
    # 将DATE和TIME合并为一个时间列
    df['TIME'] = df['DATE'] + df['TIME'].str.zfill(6)
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y%m%d%H:%M:%S')
    df.set_index('TIME', inplace=True)
    return df

# 从数据库中读取分钟数据  
def read_from_database(stock_code:str, start_date=None, end_date=None):
    # 数据库路径
    datebase = '../database/2024'
    # 将字符串转换为时间格式
    start_date = start_date if start_date else '20240101'
    end_date = end_date if end_date else '20241231'
    # 读取数据
    data = pd.read_csv(datebase + '/' + stock_code + '.csv')
    # 将时间列转换为时间格式
    data['trading_time'] = pd.to_datetime(data['trading_time'], format='%Y%m%d%H%M%S%f')
    # 将时间列设置为索引
    data.set_index('trading_time', inplace=True)
    # 筛选数据
    data = data.loc[(data.index >= pd.to_datetime(start_date)) & (data.index <= pd.to_datetime(end_date))]
    return data

In [5]:
'''
策略函数，用于生成因子值进行选股
'''
def RVar(data, winLen, log_label=True):
    '''
        出处：《1、2018-11-03_海通证券_金融工程_高频量价因子在股票与期货中的表现.pdf》第7页；
        RVar高频已实现方差
        公式：(-1 * correlation(rank(delta(log(volume), 2)), rank(((close - open) / open)), 6))
    :param data, Series: 生成因子的原始数据
    :param winLen, int: 窗长；
    :param log_label, bool: 是否取对数；
    :return: res, Series: 因子的值
    '''
    # # 对日期索引进行排序；如果日期列不是索引，需要对日期列进行排序；
    # data = data.sort_index()
    if log_label:
        # 计算取对数收益率的每日因子值
        res_RVar = data.groupby(pd.to_datetime(data.index).date).apply(lambda x: np.sum(np.square(np.log(x / x.shift(1)).replace([np.inf, -np.inf], 0))))
        # res_RVar2 = data.groupby(pd.to_datetime(data.index).date).apply(lambda x: np.sum(np.square(np.log(x / x.shift(1))[~np.isinf(np.log(x / x.shift(1)))])))   # 计算方法2
        # res_RVar3 = data.groupby(pd.to_datetime(data.index).date).apply(lambda x: np.sum(np.square(log_retRatio(x)[~np.isinf(log_retRatio(x))])))     # 计算方法3
    else:
        # 计算普通收益率的每日因子值
        res_RVar = data.groupby(pd.to_datetime(data.index).date).apply(lambda x: np.sum(np.square(x.pct_change().replace([np.inf, -np.inf], 0))))
    # 滚动计算窗长内因子值的平均值
    res_RVar = res_RVar.rolling(window=winLen).mean()
    # 日期索引格式化为时间戳格式（所有日期都要求统一为datetime.datetime 时间戳格式）
    res_RVar.index = pd.to_datetime(res_RVar.index)

    return res_RVar

In [6]:
'''
交易相关订单函数
'''
# 单只股票买卖订单基础函数
def _order(stock_code, price, volume):
    # 现金不足
    if context.portfolio.available_cash - volume * price * (1 + context.run_params.commission_rate) < 0:
        volume = int(context.portfolio.available_cash / (price * (1 + context.run_params.commission_rate)) / 100) * 100
        # print(f"现金不足，已调整为{volume}")
        
    # 100的倍数
    if volume % 100 != 0:
        volume = int(volume / 100) * 100
        # print(f"不是100的倍数, 已调整为{volume}")
            
    # 卖出数量超过持仓数量
    if context.portfolio.positions.get(stock_code, 0) < -volume:
        volume = -context.portfolio.positions.get(stock_code, 0)
        # print(f"卖出数量不能超过持仓数量, 已调整为{volume}")
        
    # 将买卖股票数量存入持仓标的信息
    context.portfolio.positions[stock_code] = context.portfolio.positions.get(stock_code, 0) + volume
    # # 如果持仓数量为0，删除该标的
    # if context.portfolio.positions[stock_code] == 0:
    #     context.portfolio.positions.pop(stock_code)
    
    # 剩余资金
    if volume > 0:
        context.portfolio.available_cash -= volume * price * (1 + context.run_params.commission_rate)
    elif volume < 0:
        context.portfolio.available_cash -= volume * price * (1 - context.run_params.commission_rate)
    
   
    # 交易记录
    context.trade_book[stock_code] = pd.concat([context.trade_book.get(stock_code, pd.DataFrame()), pd.DataFrame({'price': price, 'volume': volume}, index=[context.current_dt])])
    # 返回实际成交量
    return volume


# 定制化订单函数
# 按股数下单
def order_by_volume(stock_code, volume):
    # 获取当前股票价格
    if context.run_params.run_type == 'paper_trade':
        price = get_tick_data(stock_code)['PRICE'].values[0]
    elif context.run_params.run_type == 'backtest':
        # print(stock_code, context.current_dt)
        price = read_from_database(stock_code, context.current_dt, context.current_dt)['open'].values[0]
    volume = _order(stock_code, price, volume)
    return volume

# 目标股数下单
def order_target_volume(stock_code, volume):
    if volume < 0:
        print("数量不能为负,已调整为0")
        volume = 0
    # 当前持有数量
    hold_volume = context.positions.get(stock_code, 0)
    # 交易数量
    delta_volume = volume - hold_volume
    # 获取当前股票价格
    if context.run_params.run_type == 'paper_trade':
        price = get_tick_data(stock_code)['PRICE'].values[0]
    elif context.run_params.run_type == 'backtest':
        price = read_from_database(stock_code, context.current_dt, context.current_dt)['open'].values[0]
    volume = _order(stock_code, price, delta_volume)
    return volume

# 按价值下单
def order_by_value(stock_code, value):
    # 获取当前股票价格
    if context.run_params.run_type == 'paper_trade':
        price = get_tick_data(stock_code)['PRICE'].values[0]
    elif context.run_params.run_type == 'backtest':
        price = read_from_database(stock_code, context.current_dt, context.current_dt)['open'].values[0]
    volume = _order(stock_code, price, int(value / price))
    return volume
    
# 目标价值下单
def order_target_value(stock_code, value):
    if value < 0:
        print("价值不能为负,已调整为0")
        value = 0
    # 获取当前股票价格
    if context.run_params.run_type == 'paper_trade':
        price = get_tick_data(stock_code)['PRICE'].values[0]
    elif context.run_params.run_type == 'backtest':
        price = read_from_database(stock_code, context.current_dt, context.current_dt)['open'].values[0]
    hold_value = context.portfolio.positions.get(stock_code, 0) * price
    delta_value = value - hold_value
    volume = order_by_value(stock_code, delta_value)
    return volume

# 根据下单类型，批量下单
def order_batch(stock_codes, targets, order_type='volume'):
    for stock_code, target in zip(stock_codes, targets):
        if order_type == 'volume':
            order_by_volume(stock_code, target)
        elif order_type == 'target_volume':
            order_target_volume(stock_code, target)
        elif order_type == 'value':
            order_by_value(stock_code, target)
        elif order_type == 'target_value':
            order_target_value(stock_code, target)


# 算法交易下单函数
# twap下单
# twap_param为拆分份数
# twap_gap为拆分间隔分钟数
def order_twap(stock_codes, volumes, twap_shares=16, twap_gap=15):
    target_volume = [int(volume / twap_shares) for volume in volumes]
    # 已成交量
    traded_volume = [0] * len(stock_codes)
    # 下单twap_param次
    for i in range(twap_shares):
        for j in range(len(stock_codes)):
            # 下单
            order_volume = order_by_volume(stock_codes[j], target_volume[j])
            # 已成交数量
            traded_volume[j] += order_volume
            # 重新计算下单数量
            target_volume[j] = int((volumes[j] - traded_volume[j]) / (twap_shares - i - 1)) if twap_shares - i - 1 != 0 else volumes[j] - traded_volume[j]
        # 更新时间或休眠
        if context.run_params.run_type == 'paper_trade':
            time.sleep(twap_gap*60)
        elif context.run_params.run_type == 'backtest':
            context.current_dt += pd.Timedelta(minutes=twap_gap)
            if context.current_dt.hour == 11 and context.current_dt.minute == 45:
                context.current_dt += pd.Timedelta(hours=1, minutes=30)

# vwap下单
# vwap_shares为拆分份数
# back_days为回看天数
# vwap_gap为拆分间隔分钟数

# 获取同时段前n个交易日的数据
def get_port_mean(stock_code, curr_time, vwap_shares=16, back_days=20, vwap_gap=15):
    # 日期格式转换
    curr_time = pd.to_datetime(curr_time, format='%Y%m%d')
    # 获取回看时间
    past_time = curr_time - pd.tseries.offsets.BDay(back_days)
    # 从数据库中读取回看窗口期间数据
    past_volume_1min = read_from_database(stock_code, past_time, curr_time)['volume']
    # 将1min数据转换为vwap_gap分钟数据
    past_volume_resample = past_volume_1min.resample(str(vwap_gap) +'T').sum().to_frame()
    # 筛选数据
    past_volume_resample = past_volume_resample.loc[past_volume_resample.index.isin(past_volume_1min.index)]
    # 新建日期列用于分组计算同一日期的交易量占比
    past_volume_resample['date'] = past_volume_resample.index.date
    past_volume_resample['port'] = past_volume_resample['volume'] / past_volume_resample.groupby('date')['volume'].transform('sum')
    # 新建时间列用于分组计算同一时间段的交易量占比均值
    past_volume_resample['time'] = list(zip(past_volume_resample.index.hour, past_volume_resample.index.minute))
    past_volume_resample['port_mean'] = past_volume_resample.groupby('time')['port'].transform('mean')
    # 由于resample的特性，需要将最后一分钟的交易量占比加到前一个时间段
    past_volume_resample.iloc[vwap_shares-1, -1] += past_volume_resample.iloc[vwap_shares, -1] 
    # 返回vwap_shares个时间段的交易量占比均值
    return past_volume_resample.iloc[:vwap_shares, -1]

def order_vwap(stock_codes, volumes, vwap_shares=16, back_days=20, vwap_gap=15):
    # 计算当前时间前若干（默认20）个交易日的同时间段的交易量占比，作为拆分交易份额
    df_port_mean = pd.DataFrame(columns=stock_codes, index=range(vwap_shares))
    for stock_code in stock_codes:
        df_port_mean[stock_code] = get_port_mean(stock_code, context.current_dt, vwap_shares, back_days, vwap_gap).values
    traded_volume = [0] * len(stock_codes)
    delta_volume = [0] * len(stock_codes)
    # print(df_port_mean[stock_codes[0]])
    # 交易vwap_shares次
    for i in range(vwap_shares):
        # 交易每只股票
        for j in range(len(stock_codes)): 
            # 计算目标交易量：vwap_shares个时间段的交易量占比均值 * 总交易量 + 上次的未成交量
            target_volume = volumes[j] * df_port_mean.loc[i, stock_codes[j]] + delta_volume[j]
            # print(target_volume)
            # 下单
            order_volume = order_by_volume(stock_codes[j], target_volume)
            # 计算本次已成交数量
            traded_volume[j] += order_volume
            # 计算本次未成交数量，计入下次交易目标量
            delta_volume[j] = target_volume - order_volume
        # 更新时间或休眠
        if context.run_params.run_type == 'paper_trade':
            time.sleep(vwap_gap*60)
        elif context.run_params.run_type == 'backtest':
            context.current_dt += pd.Timedelta(minutes=vwap_gap)
            if context.current_dt.hour == 11 and context.current_dt.minute == 45:
                context.current_dt += pd.Timedelta(hours=1, minutes=30)
    # print(traded_volume)
    
    
# 总交易函数
def order(stock_codes, targets, algo_type, algo_prams, order_type):
    if algo_type == 'twap':
        order_twap(stock_codes, targets, algo_prams['twap_shares'], algo_prams['twap_gap'])
    elif algo_type == 'vwap':
        order_vwap(stock_codes, targets, algo_prams['vwap_shares'], algo_prams['back_days'], algo_prams['vwap_gap'])
    else:
        order_batch(stock_codes, targets, order_type)
            

In [7]:
# 计算vwap
def calculate_history_vwap(stock_code, start_date, end_date):
    data = read_from_database(stock_code, start_date, end_date)
    data['avg_price'] = (data['open'] + data['high'] + data['low'] + data['close']) / 4
    # 将1min频率数据转换为日频数据
    vwap = data.groupby(data.index.date).apply(lambda x: np.sum(x['avg_price'] * x['volume']) / np.sum(x['volume']))
    return vwap

In [8]:
def initialize(context):
    # 初始化运行设置
    universe = pd.read_csv('../database/000300.SH_info.csv')['con_code'].tolist()
    
    context.run_params.start_date = '20240102'
    context.run_params.end_date = '20240201'
    context.run_params.run_type = 'backtest'
    context.run_params.commission_rate = 0.0003
    context.run_params.algo_type = 'vwap'
    context.run_params.algo_params = {'vwap_shares': 16, 'back_days': 20, 'vwap_gap': 15}
    context.run_params.order_type = 'volume'
    context.benchmark = '000300.SH'
    context.universe = universe
    context.position_book = pd.DataFrame(columns=universe)

def run():
    # 初始化
    initialize(context)
    # 选股
    df_signal = pd.DataFrame(columns=context.universe)
    print('生成因子信号')
    for stock_code in tqdm(context.universe):
        series_data = read_from_database(stock_code, context.run_params.start_date, context.run_params.end_date)['close'].copy()
        df_signal[stock_code] = RVar(series_data, 20, log_label=True)
    df_position = (df_signal.shift(1).dropna().rank(axis=1, pct=True) > 0.9).astype(int)
    # print(df_position)

    # 初始化上一个交易日
    context.previous_dt = df_position.index[0]
    # print(context.previous_dt)
    # 初始化持仓记录
    context.position_book.loc[df_position.index[0], context.universe] = 0
    # print(context.position_book)
    context.total_value_book.loc[df_position.index[0], 'total_value'] = context.portfolio.total_value


    # 回测运行
    print('开始回测')
    for day in tqdm(df_position.index[1:]):
        # 更新当前时间
        context.current_dt = day + pd.Timedelta('09:30:00')
        # print(f'今天是：{context.current_dt}')
        # 获取昨日持仓
        position_yesterday = context.position_book.loc[[context.previous_dt], :]
        # 获取今日目标持仓股票
        target_stocks = df_position.loc[[day], :]
        # print(type(target_stocks))
        # 持仓股票数量
        stock_nums = 0
        # 获取今日股票价格
        for stock_code in target_stocks.columns:
            if target_stocks[stock_code].values[0] == 1:
                stock_nums += 1
                target_stocks[stock_code] *= read_from_database(stock_code, context.current_dt, context.current_dt)['open'].values[0]
        
        # print(target_stocks)
        # 计算每只股票的目标持仓金额
        target_value = context.portfolio.total_value * 0.8 / stock_nums
        # print(target_value)
        
        # # 计算每只股票的目标持仓数量
        target_stocks = target_stocks.apply(lambda x: target_value / x, axis=1).replace([np.inf, -np.inf], 0)
        # print(target_stocks)
        
        # print(f'目标持仓：{target_stocks.values}')
        # print(f'昨日持仓：{position_yesterday.values}')
        # # 根据仓位差异，计算目标交易量
        trade_stocks = pd.DataFrame(target_stocks.values - position_yesterday.values, columns=target_stocks.columns, index=target_stocks.index)
        # print(target_stocks)
        
        context.target_book = pd.concat([context.target_book, trade_stocks])
        
        # 按照交易量排序，先卖出后买入
        trade_stocks_sorted = trade_stocks.sort_values(by=day, axis=1)
        # print(trade_stocks_sorted)
        
        # 删除交易量为0的股票
        trade_stocks_sorted = trade_stocks_sorted.loc[:, trade_stocks_sorted.loc[day] != 0]
        # print(trade_stocks_sorted)
        
        # 获取交易股票及交易量列表
        trade_stocks = trade_stocks_sorted.columns.to_list()
        trade_volumes = list(trade_stocks_sorted.values[0])
        # print(f'交易列表：{trade_stocks}')
        # print(f'交易股数：{trade_volumes}')
            
        # 交易
        order(trade_stocks, trade_volumes, context.run_params.algo_type, context.run_params.algo_params, context.run_params.order_type)
        
        # 更新时间
        context.current_dt = day + pd.Timedelta('15:00:00')
        # print(context.current_dt)
        # 更新持仓记录
        context.position_book = pd.concat([context.position_book, pd.DataFrame(context.portfolio.positions, index=[day])]).fillna(0)
        
        # 更新持仓价值
        context.portfolio.position_value = 0
        for stock_code, volume in context.portfolio.positions.items():
            price = read_from_database(stock_code, context.current_dt, context.current_dt)['close'].values[0]
            context.portfolio.position_value += price * volume
            
        # 更新总资产
        context.portfolio.total_value = context.portfolio.available_cash + context.portfolio.position_value
        context.total_value_book.loc[day, 'total_value'] = context.portfolio.total_value
        
        # 交易费用分析TCA
        for stock_code, target_volume in zip(trade_stocks, trade_volumes):
            # 获取今日交易记录
            df_trade_book = context.trade_book[stock_code]
            df_trade_today = df_trade_book.loc[df_trade_book.index.date == day.date(), :]
            
            # 计算平均成交价
            # print(stock_code, target_volume, day)
            avg_price = df_trade_book.groupby(df_trade_book.index.date).apply(lambda x: np.sum(x['price'] * x['volume'].abs()) / np.sum(x['volume'].abs()) if np.sum(x['volume']) != 0 else 0).values[0]
            # 估算vwap
            vwap = calculate_history_vwap(stock_code, day, context.current_dt)
            
            # 计算今日成交量
            traded_volume = df_trade_today['volume'].sum()
            # print(traded_volume)
            # 计算交易手续费用
            fees = np.sum(df_trade_today['price'] * df_trade_today['volume'].abs() * context.run_params.commission_rate)
            
            # 获取今日开盘，收盘价
            open_time = day + pd.Timedelta(hours=9, minutes=30)
            close_time = day + pd.Timedelta(hours=15)
            today_open = read_from_database(stock_code, open_time, open_time)['open'].values[0]
            today_close = read_from_database(stock_code, close_time, close_time)['close'].values[0]
            
            # 计算交易成本
            trade_related_cost = (avg_price - today_open) * traded_volume
            opportunity_cost = (today_close - today_open) * (target_volume - traded_volume)
            
            # 总结分析结果
            tca_dict = {'avg_price': avg_price, 
                        'vwap_price': vwap,
                        'target_volume': target_volume,
                        'traded_volume': traded_volume,
                        'trade_related_cost': trade_related_cost,
                        'today_open': today_open,
                        'today_close': today_close,
                        'opportunity_cost': opportunity_cost,
                        'commission_fees': fees}
            df_tca_today = pd.DataFrame(tca_dict, index=[day])
            
            context.tca_book[stock_code] = pd.concat([context.tca_book.get(stock_code, pd.DataFrame()), df_tca_today])
        
        # 今日结束，更新时间
        context.previous_dt = day
        
    # df_strategy_return = context.total_value_book.pct_change().fillna(0)
    # df_benchmark_return = pd.read_csv('database/000300.SH.csv')['pct_chg'].fillna(0)
    
        


In [9]:
run()

生成因子信号


100%|██████████| 300/300 [00:36<00:00,  8.33it/s]


开始回测


100%|██████████| 1/1 [01:15<00:00, 75.08s/it]


In [10]:
context.tca_book['600010.SH'].to_csv('tca_book.csv')

In [ ]:
context.position_book.to_csv('position_book.csv')

In [ ]:
import quantstats as qs
qs.extend_pandas()

df_strategy_return = context.total_value_book['total_value'].pct_change().fillna(0)
df_benchmark = pd.read_csv('../database/2024/000300.SH.csv', index_col='trade_date', parse_dates=True)
df_benchmark_return = df_benchmark.loc[context.run_params.start_date:context.run_params.end_date, 'pct_chg'].fillna(0) / 100
qs.reports.full(df_strategy_return, benchmark=df_benchmark_return)
qs.reports.metrics(df_strategy_return, benchmark=df_benchmark_return)